# Domain Definitions

In [1]:
import pandas as pd
from dataclasses import dataclass
from typing import Optional, List, Set, Sequence, Dict, Tuple

In [2]:
@dataclass(frozen=True)
class Item:
    name: str
    value: float
    weight: int
    time_to_steal: int

In [3]:
@dataclass(frozen=False)
class Node:
    name: str
    items: Set[Item]

    def __init__(self, name: str):
        self.name = name
        self.items = set()

    def add_item(self, name: str, value: float, weight, tts: int) -> 'Node':
        item = Item(name, value, weight, time_to_steal=tts)
        return self._add_item(item)

    def _add_item(self, item: Item) -> 'Node':
        self.items.add(item)
        return self

    def __hash__(self) -> int:
        return self.name.__hash__()

In [4]:
@dataclass(frozen=True)
class Edge:
    source: Node
    destination: Node
    travelling_time: int
    travelling_cost: float

In [5]:
class Graph:
    nodes: Dict[str,Node]
    edges: Dict[Tuple[Node,Node], Edge]
    connections: Dict[Node, Set[Edge]]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()
        self.connections = dict()

    def get_node(self, name: str) -> Optional[Node]:
        return self.nodes.get(name)

    def get_nodes(self) -> List[Node]:
        return list(self.nodes.values())

    def get_edges(self) -> List[Edge]:
        return list(self.edges.values())

    def get_connections_from_node(self, node: Node) -> List[Edge]:
        return self.connections.get(node, [])

    def get_connections_from(self, node_name: str) -> List[Edge]:
        node: Optional[Node] = self.get_node(node_name)
        return self.get_connections_from_node(node)

    def add_edge(self, src: str, dst: str, tt: int, tc: float) -> 'Graph':
        source = Node(src)
        destination = Node(dst)
        edge = Edge(source, destination, travelling_time=tt, travelling_cost=tc)
        self._add_node(source)._add_node(destination)
        self.edges[(source, destination)] = edge
        self.connections[source] = self.connections.get(source, []) + [edge]
        return self

    def add_node(self, node_name: str) -> 'Graph':
        self.nodes[node_name] = Node(node_name)
        return self

    def _add_node(self, node: Node) -> 'Graph':
        self.nodes[node.name] = node
        return self

# Graph Construction

In [6]:
travelling_map = "~/Vault/senac/artificial-intelligence-101/datasets/travelling_map.csv"
treasure_map = "~/Vault/senac/artificial-intelligence-101/datasets/treasure_map.csv"

In [7]:
g = Graph()

In [8]:
travels = pd.read_csv(travelling_map)
treasures = pd.read_csv(treasure_map)

In [9]:
travels.head()

,source,destination,travelling_time,travelling_cost
0,Escondidos,Santa Paula,6,780
1,Escondidos,Campos,5,350
2,Escondidos,Riacho de Fevereiro,1,65
3,Escondidos,Algas,7,658
4,Escondidos,Além-do-Mar,5,535


In [10]:
travels.apply(lambda row: g.add_edge(row['source'],
                                     row['destination'],
                                     row['travelling_time'],
                                     row['travelling_cost']), axis=1)

nodes = treasures.apply(lambda row: g.get_node(row['city'])
                                     .add_item(row['name'],
                                               row['value'],
                                               row['weight'],
                                               row['tts']), axis=1)

# Problem Solving 